In [15]:
# ensure that the imported .py file will get auto imported and updated whenever there is a change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
!pip install PyMuPDF

You should consider upgrading via the 'c:\users\wang zihan\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [17]:

import fitz
import os

In [18]:

project_path = os.getcwd()
project_data_path = project_path + "/data"
project_processed_data_path = project_path + "/processed"
if not os.path.exists(project_data_path):
    os.makedirs(project_data_path)
if not os.path.exists(project_processed_data_path):
    os.makedirs(project_processed_data_path)

In [19]:
import nltk
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from nltk.corpus import stopwords
import re
import pandas as pd
pd.options.display.max_colwidth = 1000
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

import processing_pdf
pd.options.display.max_colwidth = 2000

[nltk_data] Downloading package stopwords to C:\Users\Wang
[nltk_data]     Zihan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Wang
[nltk_data]     Zihan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Wang
[nltk_data]     Zihan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
# pdf_file =  "An Empirical Survey on Long Document Summarization.pdf"
# pdf_file = "1901.00009v1.pdf"
# pdf_file = "1901.00936v3.pdf"
# pdf_file = "an image is worth 16 by 16 words.pdf"
pdf_file = "1901.00039v2.pdf"
doc, total_text, total_pages = processing_pdf.open_file(project_data_path + "/" + pdf_file)
table_of_content = doc.get_toc()
print("Auto generated table of content:")
display(table_of_content)

# some papers have not table of content
if len(table_of_content) == 0:
    print("The paper has not table of content. Need to use regular expression to map table of content.")
    res = processing_pdf.auto_find_toc(doc)
    print("Not satified the generated result or want to adjust? Build your own table of content by using this template:")
    display(res)



Auto generated table of content:


[[1, 'I Introduction', 1],
 [1, 'II Related work', 2],
 [1, 'III Our proposed method', 2],
 [2, 'III-A Density map estimation', 2],
 [2, 'III-B Method overview', 3],
 [2, 'III-C Backbone sub-network', 3],
 [2, 'III-D Mask prediction branch', 3],
 [2, 'III-E Mask-aware density density regressor', 4],
 [2, 'III-F Implementation details', 5],
 [1, 'IV Experiment', 5],
 [2, 'IV-A Evaluation metrics ', 5],
 [2, 'IV-B Datasets', 5],
 [2, 'IV-C Analysis of the proposed approaches', 6],
 [2, 'IV-D Ablation study', 6],
 [2, 'IV-E Comparison with the state-of-the-art', 8],
 [1, 'V Conclusion', 8],
 [1, 'References', 9]]

In [21]:
#uncomment this list to customize table-of-content
# table_of_content = [[1, 'I. INTRODUCTION', 1],
#  [1, 'II. SFC BASED ON IPV6 SEGMENT ROUTING', 2],
#  [1, 'III. DESIGN OF THE SRV6 PROXY', 4],
#  [2, 'A. General Concepts and State-of-the-art', 4],
#  [2, 'B. SRNKv1', 5],
#  [2, 'C. SRNKv2', 7],
#  [2, 'D. Implementation of other SR proxy types', 8],
#  [1, 'IV. TESTING ENVIRONMENT', 8],
#  [1, 'V. PERFORMANCE ANALYSIS', 9],
#  [1, 'VII. CONCLUSIONS', 11]]

# separate content into sections
processing_pdf.clear_processed_folder(project_processed_data_path)
title, authors, other_info, abstract = processing_pdf.find_meta_data(doc, table_of_content)
df_meta = pd.DataFrame([title, abstract]).T
df_meta.columns = ["Title", "Abstract"]
ds, json_dict = processing_pdf.separate_content(total_text, table_of_content)
processing_pdf.save_dataframe(ds, df_meta, json_dict, project_processed_data_path,  pdf_file.rsplit(".", 1)[0])
# extract images
processing_pdf.find_images(doc, table_of_content, total_pages, project_processed_data_path)

# display(ds)
print(f"\nTitle: {title}")
print(f"\nAuthors:{authors}")
print(f"\nOther info: {other_info}")
print(f"\nAbstract:{abstract}")


starting looking for all the sections according to the provided section title info...
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2.csv
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2_meta.csv
save the dataframe to c:\Academics\MSBA Semester 2\CS5242\Project\PDF_conversion/processed/1901.00039v2.json

Title: Mask-aware networks for crowd counting

Authors:Shengqin Jiang, Xiaobo Lu, Yinjie Lei, Lingqiao Liu

Other info: 

Abstract:—Crowd counting problem aims to count the numberof objects within an image or a frame in the videos and is usuallysolved by estimating the density map generated from the objectlocation annotations. The values in the density map, by nature,take two possible states: zero indicating no object around, anon-zero value indicating the existence of objects and the valuedenoting the local object density. In contrast to traditional meth-ods which do not diff

In [22]:
import json

# open json file
with open(project_processed_data_path + "/1901.00039v2.json") as f:
    data = json.load(f)
    for item in data:
        display(item)

{'Section_Num': 'I',
 'Section': 'I Introduction',
 'Text': 'c rowd counting is a signicant topic for crowd un- derstanding and analysis , which has attracted much attention in multimedia and computer vision community due to large and practical demands for better management, safety and security , . it aims to count the number of objects within an image or a frame in the videos and is a very challenging problem because the objects-of-interest, e.g., people, can occur at a variety of scales, with heavy occlusions and cluttered visual appearances. also, due to the difculty in providing highly detailed annotations such as object bounding boxes or instance-level segmentation masks, existing datasets usually adopt a weak-level annotating scheme by labeling each object with a dot inside. these challenges make the traditional detection based approach less robust and most existing methods choose to solve this problem by estimating a density map generated from the dot-level annotation. once the 

{'Section_Num': 'II',
 'Section': 'II Related work',
 'Text': 'to date, many approaches have been proposed to study the issues existing in crowd counting , . here we present a brief review of the related work. for a more detailed survey of crowd counting, we refer the readers to , , . detection seems to be a straightforward solution for crowd counting. the most early methods use hand-crafted features such as haar wavelets , histogram oriented gradients to model the pedestrian, which are then fed to classiers to distinguish whether there is pedestrian or not. initially, , studied the monocular pedestrian detection by a diverse set of low-level feature-based systems. although monocular- based methods work well in a low density region, the perfor- mance is severely affected when they meet the crowded scenes with occlusion and scene clutter. to further consider this issue, more information of the pedestrian is taken into account. zhao et al. used multiple partially occluded human hypothese

{'Section_Num': 'III',
 'Section': 'III Our proposed method',
 'Text': 'a. density map estimation before elaborating the design of our network, we rst briey introduce the creation of the ground-truth density maps and the related training losses. this paper considers the case that a point-wise annotation is provided for training images. specically, a dot is annotated within each object-of-interest, i.e., people head. this pointwise annotation is further converted into a density map: d = x xia g(x xi ), ieee transactions on circuits and systems for video technology fig. an overview of our proposed method. it contains three modules: the backbone, the mask prediction branch and the mask-aware density regression branch. where x rdenotes the image coordinate and xi denotes the annotated head location. g( xxi ) denotes a gaussian kernel with xi as the mean vector and as the empirically chosen variance term. a typical choice of will make g( xxi ) = if x is not within the local neighborhood of 

{'Section_Num': 'IV',
 'Section': 'IV Experiment',
 'Text': 'in this section, we conduct experiments on three challenging public datasets to demonstrate the effectiveness our proposed method including shanghaitech dataset , ucf cc dataset and worldexpo dataset . the purposes of our experiments are threefold: verify if the proposed mask-aware strategies lead to signicant improvement over the baselines. identify the most effective mask-aware den- sity estimation solution. compare our proposed approach against the state-of-the-art methods. in our experiments, we use shanghaitech dataset a to achieve the rst and the second objective. the identied best-performed solution will then be compared against the state-of-the-art on all three datasets. in what follows, we present the evaluation criterion and datasets in our experiments. then we present a detailed anal- ysis of the proposed solutions and identify the most effective one. finally, we compare our method with other state-of-the- art meth

{'Section_Num': 'V',
 'Section': 'V Conclusion',
 'Text': 'in this paper, we address the crowd counting problem with deep neural networks. our main discovery is the benet ieee transactions on circuits and systems for video technology table iii the performance comparison on the shanghaitech dataset. part a part b method mae mse mae mse cc-counting fcn mcnn tdf-net switching-net ba-cnn netvlad cp-cnn csrnet our method our model csr table iv the performance comparison on the ucf cc dataset. method mae mse cc-counting fcn mcnn tdf-net switching-net ba-cnn netvlad cp-cnn csrnet our method our model csr table v the performance comparison on the worldexpo dataset. method s s s s s avg cc-counting mcnn tdf-net switching-net ba-cnn netvlad cp-cnn csrnet our method our model csr of using a dedicated network branch to predict the fore- ground/background mask and incorporating mask prediction into density map estimation. we systematically study ve different designs of the mask-aware density estima

{'Section_Num': 'References',
 'Section': 'References',
 'Text': '] wongun choi and silvio savarese, understanding collective activitiesof people from videos, ieee transactions on pattern analysis and machine intelligence, vol. , no. , pp. , jing shao, chen change loy, kai kang, and xiaogang wang, crowded scene understanding by deeply learned volumetric slices, ieee trans- actions on circuits and systems for video technology, vol. , no. , pp. , siyu huang, xi li, zhongfei zhang, fei wu, shenghua gao, rongrong ji, and junwei han, body structure aware deep crowd counting, ieee transactions on image processing, vol. , no. , pp. , yuke li, a deep spatiotemporal perspective for understanding crowd behavior, ieee transactions on multimedia, cong zhang, kai kang, hongsheng li, xiaogang wang, rong xie, and xiaokang yang, data-driven crowd understanding: a baseline for a large-scale crowd dataset, ieee transactions on multimedia, vol. , no. , pp. , vishwanath a sindagi and vishal m patel, a sur

{'Section_Num': 'No_title',
 'Section': 'No_title',
 'Text': 'ieee transactions on circuits and systems for video technology mask-aware networks for crowd counting shengqin jiang, xiaobo lu, yinjie lei, lingqiao liu abstractcrowd counting problem aims to count the number of objects within an image or a frame in the videos and is usually solved by estimating the density map generated from the object location annotations. the values in the density map, by nature, take two possible states: zero indicating no object around, a non-zero value indicating the existence of objects and the value denoting the local object density. in contrast to traditional meth- ods which do not differentiate the density prediction of these two states, we propose to use a dedicated network branch to predict the object/non-object mask and then combine its prediction with the input image to produce the density map. our rationale is that the mask prediction could be better modeled as a binary segmentation problem a